In [1]:
import os, os.path as op
import glob
import argparse, sys, copy, time
import matplotlib.pyplot as plt
import numpy as np, pandas as pd
# import seaborn as sns
import cProfile
import pstats
from datetime import datetime

%pdb off
%matplotlib notebook

Automatic pdb calling has been turned OFF


#### Setup Structure

In [4]:
## you will need to edit this
path_AT  = op.join(op.expanduser('~'), 'ARIA-tools_git')

In [5]:
# get the git branch
os.chdir(path_AT)
gbranch = !git branch --show-current
gbranch = gbranch[0]
print ('On branch:', gbranch)

On branch: Profiling


In [6]:
# setup directory for aria outputs and profiles
track   = '004'

path_wd = op.join(op.expanduser('~'), f'Virtual_Tests_{track}') 
os.makedirs(path_wd, exist_ok=True)
os.chdir(path_wd)

path_profs = op.join(path_wd, 'Profiles')
os.makedirs(path_wd, exist_ok=True)

print ('Working directory:', path_wd)

Working directory: /home/jovyan/Virtual_Tests_004


In [7]:
path_dem  = op.join(path_wd, 'DEM', 'SRTM_3arcsec.dem')
bbox      = '36.75 37.225 -76.655 -75.928' # Hampton Roads
n_threads = 4

In [8]:
def clear_directory(path):
    """ Start from a clean directory """
    files = os.listdir(path)
    files = [op.join(path, f) for f in files]
    # keep the DEM as it saves some time
    if 'DEM' in files:
        files.remove('DEM')
        
    !rm -rf {' '.join(files)}
    return

## Single Frame

In [9]:
# 18 products between these dates for a single frame
st, en, nprod = 20170201, 20170501, 18 
# st, en, nprod = 20170101, 20180101, 80 ## longer runs


path_proc = op.join(path_wd, f'1Frame_{nprod}_{gbranch}')
os.makedirs(path_proc, exist_ok=True)
os.chdir(path_proc)

In [13]:
# start from a fresh directory
clear_directory(path_proc)

print ('Ready to download ARIA to:', path_proc)

Ready to download ARIA to: /home/jovyan/Virtual_Tests_004/1Frame_18_Profiling


### Download

In [16]:
# clean out products so everything is right later on
!rm -rf {path_proc}/products

In [17]:
# download the physical products; 
# really this should count towards the time vs for virtual profiling, but it's pretty fast and makes comparison overly difficult
cmd = f'ariaDownload.py -t {track} --st {st} --en {en}  --b "{bbox}" -nt 4'
print (cmd)
!{cmd}

ariaDownload.py -t 004 --st 20170201 --en 20170501  --b "36.75 37.225 -76.655 -75.928" -nt 4
ARIA-tools Version: 1.1.0
Wrote -- 18 -- products to: /home/jovyan/Virtual_Tests_004/1Frame_18_Profiling/products


In [22]:
!which ariaDownload.py -h

/home/jovyan/.local/envs/ARIA/bin/ariaDownload.py


In [20]:
# download the text file with the virtual urls
cmd = f'ariaDownload.py -t 004 --st {st} --en {en}  --b "{bbox}" -nt 4 -o url'
print (cmd)
!{cmd}

ariaDownload.py -t 004 --st 20170201 --en 20170501  --b "36.75 37.225 -76.655 -75.928" -nt 4 -o url
Traceback (most recent call last):
  File "/home/jovyan/.local/envs/ARIA/bin/ariaDownload.py", line 4, in <module>
    __import__('pkg_resources').run_script('ARIAtools==1.1.0', 'ariaDownload.py')
  File "/home/jovyan/.local/envs/ARIA/lib/python3.10/site-packages/pkg_resources/__init__.py", line 672, in run_script
    self.require(requires)[0].run_script(script_name, ns)
  File "/home/jovyan/.local/envs/ARIA/lib/python3.10/site-packages/pkg_resources/__init__.py", line 1460, in run_script
    raise ResolutionError(
pkg_resources.ResolutionError: Script 'scripts/ariaDownload.py' not found in metadata at '/home/jovyan/.local/envs/ARIA/lib/python3.10/site-packages/ARIAtools-1.1.0.dist-info'


In [ ]:
cmd = f'ariaExtract.py -f "{path_proc}/products/*.nc" --dem Download -nt 4'
print (cmd)
!{cmd}

### tsSetup1 - Physical

In [ ]:
n     = 2 # run it twice; stats are stable

# setup the file name of the profile
today = datetime.today().strftime('%m%d') 
bname = f'PHYS_{nprod}_{today}'

In [ ]:
for nn in range(n):
    dst = op.join(path_profs, f'{bname}{nn}.profile')
                
    cmd = f'python -m cProfile -o {dst} '\
          f'-s time {path_AT}/bin/ariaTSsetup.py -f "{path_proc}/products/S1*.nc" '\
          f'--dem {path_dem} --layers "all" --bperp -nt {n_threads} '
    print (cmd)
    !{cmd}
    print ('\n-------Finished:', dst)
    clean_dir(path_proc)

### tsSetup1 - Virtual

In [ ]:
n     = 5 # run it nx

# setup the file name of the profile
today = datetime.today().strftime('%m%d') 
bname = f'VIRT_{nprod}_{today}'

In [ ]:
for nn in range(n):
    dst = op.join(path_profs, f'{bname}{nn}.profile')
                
    cmd = f'python -m cProfile -o {dst} '\
          f'-s time {path_AT}/bin/ariaTSsetup.py -f "{path_proc}/products/track004_bbox-77W36S-75E38N_0.txt" '\
          f'--dem {path_dem} --layers "all" --bperp -nt {n_threads} '
    print (cmd)
    !{cmd}
    print ('\n-------Finished:', dst)
    clean_dir(path_proc)

## Multiple Frames

In [ ]:
# 21 products between these dates for multiple frames
st_tr, en_tr, nprod_tr = 20190101, 20190220, 21
# st_tr, en_tr, nprod_tr  = 20170101, 20170315, 83 

path_proc = op.join(path_wd, f'Track_{nprod_tr}_{gbranch}')
os.makedirs(path_proc, exist_ok=True)
os.chdir(path_proc)

In [ ]:
# start from a fresh directory
clear_directory(path_res)

print ('Ready to download ARIA to:', path_proc)

### Download

In [ ]:
# download the physical products; 
# really this should count towards the time vs for virtual profiling, but it's pretty fast and makes comparison overly difficult
cmd = f'ariaDownload.py -t {track} --st {st_tr} --en {en_tr} -nt 4'
print (cmd)
!{cmd}

In [ ]:
# download the text file with the virtual urls
cmd = f'ariaDownload.py -t 004 --st {st_tr} --en {en_tr} -nt 4 -o url'
print (cmd)
!{cmd}

In [ ]:
cmd = f'ariaExtract.py -f "{path_proc}/products/*.nc" --dem Download -nt 4'
print (cmd)
!{cmd}

### tsSetup+ - Physical

In [ ]:
n     = 2 # run it twice; stats are stable

# setup the file name of the profile
today = datetime.today().strftime('%m%d') 
bname = f'PHYS_{nprod_tr}_{today}'

In [ ]:
for nn in range(n):
    dst = op.join(path_profs, f'{bname}{nn}.profile')
                
    cmd = f'python -m cProfile -o {dst} '\
          f'-s time {path_AT}/bin/ariaTSsetup.py -f "{path_proc}/products/S1*.nc" '\
          f'--dem {path_dem} --layers "all" --bperp -nt {n_threads} '
    print (cmd)
    !{cmd}
    print ('\n-------Finished:', dst)
    clean_dir(path_proc)

### tsSetup+ - Virtual

In [ ]:
n     = 5 # run it nx

# setup the file name of the profile
today = datetime.today().strftime('%m%d') 
bname = f'VIRT_{nprod}_{today}'

In [ ]:
for nn in range(n):
    dst = op.join(path_profs, f'{bname}{nn}.profile')
                
    cmd = f'python -m cProfile -o {dst} '\
          f'-s time {path_AT}/bin/ariaTSsetup.py -f "{path_proc}/products/track004_0.txt" '\
          f'--dem {path_dem} --layers "all" --bperp -nt {n_threads} '
    print (cmd)
    !{cmd}
    print ('\n-------Finished:', dst)
    clean_dir(path_proc)